In [478]:
import numpy as np
import pandas as pd
import cobra.test
import os

from os.path import join

from time import time


from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

a = cobra.io.read_sbml_model("iEC1344_C.xml")

In [479]:
i= 0
r = a.reactions[0]
l2= list()
l3= list()

for r in a.reactions:            #separates reactions that have biomass values for positive and negative flux values
    
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))

    i3= 101

    while i3>100 and i3<len(csvFile):
        inf= csvFile.iloc[i3,  2]
        flt= float(csvFile.iloc[i3,  2])
    
        if csvFile.iloc[i3,  2] == flt and r_id not in l2:
            l2.append(r_id)     #l2 contains reactions that have biomass values for positive flux values
        i3= i3+1
    
    i4= 0
    
    while i4<100:
        inf= csvFile.iloc[i4,  2]
        flt= float(csvFile.iloc[i4,  2])
    
        if csvFile.iloc[i4,  2] == flt and r_id not in l3:
            l3.append(r_id)    #l3 contains reactions that have biomass values for negative flux values
        i4= i4+1

    i= i+1

D1= pd.DataFrame (l2, columns = ['Reaction names'])
D2= pd.DataFrame (l3, columns = ['Reaction names'])
b= D1.iloc[:, 0]
c= D2.iloc[:, 0]

union = pd.Series(np.union1d(b, c))
intersect = pd.Series(np.intersect1d(b, c))
bigb2= union[~union.isin(intersect)]

D3= pd.DataFrame (bigb2, columns = ['Reaction names'])
D3
D3.to_csv("res2.csv")      #contains reactions that have biomass values for either positive or negative flux values

In [480]:
i= 0
diff_saturated= list()
diff_indecr= list()
flat= list()
r = a.reactions[0]


for r in a.reactions:    #separates everything that has an increasing/decreasing pattern from everything that has a saturating pattern and everything that is flat
    l= list()
    n=0
    n2=0
    count=0
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))

    while n < 201:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    d= pd.DataFrame (l, columns = ['Biomass'])


    if len(d)> 1:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.02 and d.iloc[n2,  0] < max(d["Biomass"])+0.02:
                count= count+1
    
            n2= n2+1
        
        if count> 0.8*len(d) and count< 0.95*len(d):
            diff_saturated.append(r_id)

        elif count> 0.95*len(d):
            flat.append(r_id)
        
        else:
            diff_indecr.append(r_id)

    
    i= i+1

flat
diff_indecr
diff_saturated

['EX_h_e',
 'EX_h2_e',
 'EX_ppt_e',
 'EX_fe2_e',
 'EX_fe3_e',
 'ACKr',
 'ACOLIPAabctex',
 'ADA',
 'ADD',
 'ADNK1',
 'ADNUC',
 'ADPT',
 'AMPN',
 'ASPt2pp_copy2',
 'CHLabcpp',
 'CLIPAabctex',
 'CBPS',
 'COLIPAabctex',
 'DADA',
 'CU1abcpp',
 'CYSabc2pp',
 'DHAPT',
 'ECA4COLIPAabctex',
 'ENLIPAabctex',
 'F6PP',
 'FA120ACPHi',
 'FA140ACPHi',
 'FA141ACPHi',
 'FA160ACPHi',
 'FA161ACPHi',
 'FDH4pp',
 'FACOAL60t2pp',
 'ASNNpp',
 'ASNS1',
 'FEROpp',
 'FE2tex',
 'FE3tex',
 'FTHFLi',
 'ASPO3',
 'G6PP',
 'ASPO4',
 'ASPO5',
 'GLCabcpp',
 'GALabcpp',
 'GLUN',
 'GART',
 'GDPMNP',
 'GLUSy',
 'GLYCK2',
 'GLYBabcpp',
 'GLYC3Pabcpp',
 'GLYCLTDy',
 'HXCT',
 'HXPRT',
 'Htex',
 'GLYCTO2',
 'HEX1',
 'HEX7',
 'HG2abcpp',
 'INSH',
 'INSK',
 'GLYOX3',
 'L_LACD2_copy1',
 'L_LACD3_copy1',
 'LIPAabctex',
 'ME1',
 'MLTG3',
 'MLTG4',
 'MLTG5',
 'MMM',
 'MN6PP',
 'MG2uabcpp',
 'MICITDr_copy1',
 'NADH16pp',
 'GSNK',
 'NTD7',
 'NTD8',
 'NO3R2bpp',
 'NTD9',
 'NTPTP1',
 'NTPTP2',
 'NTRIR3pp',
 'GTPDPDP',
 'NTRIR4pp',
 'OA

In [481]:
flat_only_above_zero= list()
diff_indecr_only_above_zero= list()
diff_saturated_only_above_zero= list()

i=0
r_id = l2[i]

while i< len(l2):    #finds reactions with an increasing/decreasing pattern above zero
    r_id = l2[i]
    if r_id in diff_indecr and r_id not in l3 and r_id not in diff_indecr_only_above_zero:
        diff_indecr_only_above_zero.append(r_id)
    i= i + 1

diff_indecr_only_above_zero

i=0
r_id = l2[i]

while i< len(l2):   #finds reactions with a flat pattern above zero
    r_id = l2[i]
    if r_id in flat and r_id not in l3 and r_id not in flat_only_above_zero:
        flat_only_above_zero.append(r_id)
    i= i + 1

flat_only_above_zero

i=0
r_id = l2[i]

while i< len(l2):  #finds reactions with a saturating pattern above zero
    r_id = l2[i]
    if r_id in diff_saturated and r_id not in l3 and r_id not in diff_saturated_only_above_zero:
        diff_saturated_only_above_zero.append(r_id)
    i= i + 1

diff_saturated_only_above_zero

['SPODM']

In [482]:
int_reactions= list()
i= -1
r = a.reactions[0]
r_id = r.id
zero= list()
one= list()
one_at_zero= list()

for r in a.reactions:  #finds reactions that have one feasible value at flux=0 or for another flux
    i= i+1
    r = a.reactions[i]
    r_id = r.id

    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    v= len(csvFile["Biomass"])-csvFile["Biomass"].isna().sum()

    flt= float(csvFile.iloc[100,  2])
    
    if v == 1 and csvFile.iloc[100,  2] != flt:
        one.append(r_id)
    elif v == 0:
        zero.append(r_id)
        
    if v == 1 and csvFile.iloc[100,  2] == flt:
        one_at_zero.append(r_id)

print(one)
print(zero)
print(one_at_zero)

D= pd.DataFrame (one, columns = ['1 feasible value'])
D_at_zero= pd.DataFrame (one_at_zero, columns = ['1 feasible value_at_zero'])

[]
[]
['EX_cm_e', 'EX_cobalt2_e', 'EX_colipa_e', 'EX_colipap_e', 'EX_crn_e', 'EX_crn__D_e', 'DM_4crsol_c', 'EX_cu2_e', 'DM_5drib_c', 'EX_cyan_e', 'DM_aacald_c', 'DM_amob_c', 'DM_mththf_c', 'EX_glyb_e', 'EX_gthox_e', 'BIOMASS_Ec_iJO1366_WT_53p95M', 'BIOMASS_Ec_iJO1366_core_53p95M', 'EX_14glucan_e', 'EX_15dap_e', 'EX_h2o2_e', 'EX_hacolipa_e', 'EX_halipa_e', 'EX_34dhpac_e', 'EX_hg2_e', 'EX_his__L_e', 'EX_4hoxpacd_e', 'EX_5mtr_e', 'EX_cys__D_e', 'EX_ile__L_e', 'EX_acgal_e', 'EX_inost_e', 'EX_k_e', 'EX_kdo2lipid4_e', 'EX_acolipa_e', 'EX_adocbl_e', 'EX_leu__L_e', 'EX_lipa_e', 'EX_lipa_cold_e', 'EX_ag_e', 'EX_lipoate_e', 'EX_ala_B_e', 'EX_malthx_e', 'EX_maltpt_e', 'EX_malttr_e', 'EX_maltttr_e', 'EX_arbt_e', 'EX_meoh_e', 'EX_met__D_e', 'EX_aso3_e', 'EX_met__L_e', 'EX_metsox_R__L_e', 'EX_metsox_S__L_e', 'EX_mg2_e', 'EX_mincyc_e', 'EX_btn_e', 'EX_mmet_e', 'EX_mn2_e', 'EX_ca2_e', 'EX_cbi_e', 'EX_mobd_e', 'EX_n2o_e', 'EX_na1_e', 'EX_nac_e', 'EX_cbl1_e', 'EX_cd2_e', 'EX_chol_e', 'EX_cl_e', 'EX_ni2_

In [483]:
flat_only_below_zero= list()
diff_indecr_only_below_zero= list()
diff_saturated_only_below_zero= list()

i=0
r_id = l3[i]

while i< len(l3):    #finds reactions with an increasing/decreasing pattern below zero
    r_id = l3[i]
    if r_id in diff_indecr and r_id not in l2 and r_id not in diff_indecr_only_below_zero:
        diff_indecr_only_below_zero.append(r_id)
    i= i + 1

diff_indecr_only_below_zero

i=0
r_id = l3[i]

while i< len(l3):   #finds reactions with a flat pattern below zero
    r_id = l3[i]
    if r_id in flat and r_id not in l2 and r_id not in flat_only_below_zero:
        flat_only_below_zero.append(r_id)
    i= i + 1

flat_only_below_zero

i=0
r_id = l3[i]

while i< len(l3):   #finds reactions with a saturating pattern below zero
    r_id = l3[i]
    if r_id in diff_saturated and r_id not in l2 and r_id not in diff_saturated_only_below_zero:
        diff_saturated_only_below_zero.append(r_id)
    i= i + 1

diff_saturated_only_below_zero

['EX_h2_e',
 'EX_ppt_e',
 'ACOLIPAabctex',
 'CHLabcpp',
 'CLIPAabctex',
 'COLIPAabctex',
 'CU1abcpp',
 'ECA4COLIPAabctex',
 'ENLIPAabctex',
 'ASPO3',
 'ASPO4',
 'ASPO5',
 'GALabcpp',
 'GLYBabcpp',
 'HG2abcpp',
 'LIPAabctex',
 'NO3R2bpp',
 'NTRIR3pp',
 'NTRIR4pp',
 'XYLabcpp',
 'PPPGO']

In [484]:
set1= set(flat)
set2= set(flat_only_above_zero)
set3= set(flat_only_below_zero)
flat_both_positive_and_negative= list(set1-set2-set3)   #finds reactions that are flat for both above and below 0 flux values
flat_both_positive_and_negative

['ACACT7r',
 'GLCtex_copy1',
 'ACt4pp',
 'ADK3',
 'EAR120y',
 'ABUTt2pp',
 'ALATA_L',
 'GLUt2rpp',
 'GLYCK',
 'EAR140y',
 'HACD4i',
 'SERt4pp',
 'ACACT4r',
 'HACD7i',
 'L_LACtex_copy2',
 'HACD8i',
 'CTECOAI7_copy2',
 'THRA',
 'GLCtex_copy2',
 'HACD5i',
 'FMNRx2',
 'ACACT12',
 'CTECOAI6_copy2',
 'ALDD19xr_copy2',
 'ACS',
 'NAt3pp',
 'DHORDfum',
 'SSALx',
 'ACACT8',
 'MLDCP2App',
 'GLCP',
 'VPAMTr_copy2',
 'DARBt2rpp_copy2',
 'CTECOAI6_copy1',
 'HACD6',
 'ACACT8r_copy1',
 'ACACT8r_copy2',
 'CTECOAI8_copy1',
 'ALAt4pp',
 'URAt2pp_copy1',
 'EAR181y',
 'ADK1',
 'HACD3i',
 'ASPt2pp_copy1',
 'EAR121y',
 'DHORD2',
 'KAT1',
 'MICITDr_copy2',
 'EAR100y',
 'ACACT10',
 'ICHORS_copy2',
 'PROt4pp',
 'ALAt2pp_copy1',
 'HACD5',
 'GLCP2',
 'MDDCP3pp',
 'HACD1i',
 'GLYt4pp',
 'CTECOAI7_copy1',
 'FADRx2',
 'ACACT5r',
 'INDOLEt2pp',
 'HPYRRx',
 'VPAMTr_copy1',
 'GLUt4pp',
 'GLYCLTt4pp',
 'CA2t3pp',
 'CRNt8pp',
 'HACD7',
 'NDPK1',
 'EAR161y',
 'L_LACtex_copy1',
 'VALTA',
 'EAR180y',
 'TRSARr',
 'THRD',
 'E

In [485]:
i= 0
r_id= diff_saturated_only_below_zero[i]
diff_saturatedFINAL_only_below_zero= list()
Ushaped_reactions_only_below_zero= list()

for r_id in diff_saturated_only_below_zero:   #separates saturated from U-shaped reactions below zero

    l= list()
    n=0
    n2=0
    r_id = diff_saturated_only_below_zero[i]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))

    while n < 201:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    d= pd.DataFrame (l, columns = ['Biomass'])
    
    if d.iloc[0,0] > d.iloc[1,0]-0.02 and d.iloc[0,0] < d.iloc[1,0]+0.02 or\
       d.iloc[1,0] > d.iloc[0,0]-0.02 and d.iloc[1,0] < d.iloc[0,0]+0.02 or\
       d.iloc[len(d)-2,0] > d.iloc[len(d)-1,0]-0.02 and d.iloc[len(d)-2,0] < d.iloc[len(d)-1,0]+0.02 or\
       d.iloc[len(d)-1,0] > d.iloc[len(d)-2,0]-0.02 and d.iloc[len(d)-1,0] < d.iloc[len(d)-2, 0]+0.02:
        diff_saturatedFINAL_only_below_zero.append(r_id)
    
    else:
        Ushaped_reactions_only_below_zero.append(r_id)
    
    i= i+1

diff_saturatedFINAL_only_below_zero
Ushaped_reactions_only_below_zero

['EX_h2_e', 'NTRIR3pp', 'NTRIR4pp', 'PPPGO']

In [486]:
i= 0
r_id= diff_saturated[i]
diff_saturatedFINAL= list()
Ushaped_reactions= list()

for r_id in diff_saturated:

    l= list()
    n=0
    n2=0
    r_id = diff_saturated[i]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))

    while n < 201:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    d= pd.DataFrame (l, columns = ['Biomass'])
    
    if d.iloc[0,0] > d.iloc[1,0]-0.02 and d.iloc[0,0] < d.iloc[1,0]+0.02 or\
       d.iloc[1,0] > d.iloc[0,0]-0.02 and d.iloc[1,0] < d.iloc[0,0]+0.02 or\
       d.iloc[len(d)-2,0] > d.iloc[len(d)-1,0]-0.02 and d.iloc[len(d)-2,0] < d.iloc[len(d)-1,0]+0.02 or\
       d.iloc[len(d)-1,0] > d.iloc[len(d)-2,0]-0.02 and d.iloc[len(d)-1,0] < d.iloc[len(d)-2, 0]+0.02:
        diff_saturatedFINAL.append(r_id)
    
    else:
        Ushaped_reactions.append(r_id)
    
    i= i+1

set1= set(diff_saturatedFINAL)
set2= set(diff_saturatedFINAL_only_below_zero)

set3= set(Ushaped_reactions)
set4= set(Ushaped_reactions_only_below_zero)

diff_saturatedFINAL_both_positive_and_negative= list(set1-set2)   #finds saturated reactions for both above and below 0 flux values
Ushaped_reactions_both_positive_and_negative= list(set3-set4)     #fines U-shaped reactions for both above and below 0 flux values

diff_saturatedFINAL_both_positive_and_negative
Ushaped_reactions_both_positive_and_negative

['EX_fe3_e',
 'GART',
 'EX_h_e',
 'Htex',
 'SULabcpp',
 'ACKr',
 'FEROpp',
 'NADH16pp',
 'ASNS1',
 'GLUSy',
 'PRPPS',
 'NTD9',
 'GLYCK2',
 'PTAr']

In [487]:
i2= 0
r_id = diff_indecr_only_above_zero[0]
mon_incr_only_above_zero= list()
mon_decr_only_above_zero= list()

for r_id in diff_indecr_only_above_zero:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = diff_indecr_only_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True:
            mon_decr_only_above_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            mon_incr_only_above_zero.append(r_id)

    
    i2= i2 + 1

mon_decr_only_above_zero


['DM_oxam_c',
 'EX_12ppd__R_e',
 'EX_h2s_e',
 'EX_3hpp_e',
 'EX_hom__L_e',
 'EX_acser_e',
 'EX_cit_e',
 '2AGPA120tipp',
 '2AGPA140tipp',
 '2AGPA141tipp',
 '2AGPA160tipp',
 '2AGPA161tipp',
 '2AGPA180tipp',
 '2AGPA181tipp',
 '2AGPE120tipp',
 '2AGPE140tipp',
 '2AGPE141tipp',
 '2AGPE160tipp',
 '2AGPE161tipp',
 '2AGPE180tipp',
 '2AGPE181tipp',
 'EX_thym_e',
 '2AGPG120tipp',
 '2AGPG140tipp',
 '2AGPG141tipp',
 '2AGPG160tipp',
 '2AGPG161tipp',
 '2AGPG180tipp',
 '2AGPG181tipp',
 'EX_urea_e',
 '2MAHMP',
 '3HAD100',
 '3AMACHYD',
 '3HAD120',
 '3HAD121',
 '3HAD140',
 '3HAD141',
 '3HAD160',
 '3HAD161',
 'AACPS8',
 '3HAD180',
 '3HAD181',
 'AACPS9',
 'AACTOOR',
 '3HPPtpp',
 'ABTA',
 'ABUTD',
 '3OAR100',
 '3OAR120',
 '3OAR121',
 '3OAR141',
 '3OAR160',
 '3OAR161',
 '3OAR180',
 '3OAR181',
 '3OAS100',
 '3OAS120',
 '3OAS121',
 '3OAS141',
 '3OAS160',
 '3OAS161',
 '3OAS180',
 '3OAS181',
 'ACGAMK',
 '3PEPTabcpp',
 'ACM6PH',
 'ACOAD4f',
 'ACOAD5f',
 'ACOAD6f',
 'ACOAD7f',
 'ACOAD8f',
 'ACPPAT120',
 'ACPPAT140'

In [488]:
i2= 0
r_id = diff_indecr_only_below_zero[0]
mon_incr_only_below_zero= list()
mon_decr_only_below_zero= list()

for r_id in diff_indecr_only_below_zero:                 #this code separates reactions with an increasing trend from reactions with a decreasing trend below zero
    
    i=0
    r_id = diff_indecr_only_below_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True:
            mon_decr_only_below_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            mon_incr_only_below_zero.append(r_id)

    
    i2= i2 + 1

mon_decr_only_below_zero


['EX_cmp_e',
 'EX_csn_e',
 'EX_glcur_e',
 'EX_glcur1p_e',
 'EX_gln__L_e',
 'EX_glyc2p_e',
 'EX_gmp_e',
 'EX_gsn_e',
 'EX_23camp_e',
 'EX_23cgmp_e',
 'EX_23cump_e',
 'EX_2ddglcn_e',
 'EX_3amp_e',
 'EX_3cmp_e',
 'EX_3gmp_e',
 'EX_3ump_e',
 'EX_cytd_e',
 'EX_dad_2_e',
 'EX_imp_e',
 'EX_acac_e',
 'EX_damp_e',
 'EX_ins_e',
 'EX_acgam1p_e',
 'EX_lcts_e',
 'EX_adn_e',
 'EX_dcmp_e',
 'EX_dcyt_e',
 'EX_mal__D_e',
 'EX_alaala_e',
 'EX_dgmp_e',
 'EX_man6p_e',
 'EX_amp_e',
 'EX_arg__L_e',
 'EX_dgsn_e',
 'EX_dimp_e',
 'EX_din_e',
 'EX_dtmp_e',
 'EX_orn_e',
 'EX_dump_e',
 'EX_pi_e',
 'EX_duri_e',
 'EX_psclys_e',
 'EX_pser__L_e',
 'EX_r5p_e',
 'EX_rmn_e',
 'EX_ser__D_e',
 'EX_f6p_e',
 'EX_fald_e',
 'EX_thrp_e',
 'EX_thymd_e',
 'EX_tre_e',
 'EX_trp__L_e',
 'EX_ump_e',
 'EX_uri_e',
 'EX_frulys_e',
 'EX_fruur_e',
 'EX_g1p_e',
 'EX_g3pc_e',
 'EX_g3pe_e',
 'EX_g3pg_e',
 'EX_g3pi_e',
 'EX_g3ps_e',
 'EX_g6p_e',
 'EX_gal_bD_e',
 'EX_gal1p_e',
 'EX_galctn__D_e',
 'EX_galctn__L_e',
 'EX_gam6p_e',
 'DHORTS',
 '

In [489]:
i2= 0
r_id = diff_indecr[0]
mon_incr= list()
mon_decr= list()

for r_id in diff_indecr:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend
    
    i=0
    r_id = diff_indecr[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True:
            mon_decr.append(r_id)
        elif my_series.is_monotonic ==True:
            mon_incr.append(r_id)

    
    i2= i2 + 1

set1= set(mon_decr)
set2= set(mon_decr_only_above_zero)
set3= set(mon_decr_only_below_zero)
mon_decr_both_positive_and_negative= list(set1-set2-set3)

set1= set(mon_incr)
set2= set(mon_incr_only_above_zero)
set3= set(mon_incr_only_below_zero)
mon_incr_both_positive_and_negative= list(set1-set2-set3)

mon_decr_both_positive_and_negative    #finds reactions with decreasing trend for both positive and negative fluz values
mon_incr_both_positive_and_negative    #finds reactions with increasing trend for both positive and negative fluz values

['GLCptspp', 'CRNDt2rpp', 'DURAD']

In [490]:
i2=0
mon_decr_and_stable_only_above_zero= list()
mon_decr_only_only_above_zero= list()

while i2< len(mon_decr_only_above_zero):  #this code separates reactions that may have flat points from reactions that always decrease above zero
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(mon_decr_only_above_zero[i2]))
    i=0
    l= list()
    
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    boolean = csvFile2["Biomass"].duplicated().any()
    
    if boolean == True:
        mon_decr_and_stable_only_above_zero.append(mon_decr_only_above_zero[i2])
    
    if boolean == False:
        mon_decr_only_only_above_zero.append(mon_decr_only_above_zero[i2])
    
    i2= i2+1

mon_decr_and_stable_only_above_zero

['FRD3']

In [491]:
i2=0
mon_decr_and_stable_only_below_zero= list()
mon_decr_only_only_below_zero= list()

while i2< len(mon_decr_only_below_zero):  #this code separates reactions that may have flat points from reactions that always decrease below zero
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(mon_decr_only_below_zero[i2]))
    i=0
    l= list()
    
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    boolean = csvFile2["Biomass"].duplicated().any()
    
    if boolean == True:
        mon_decr_and_stable_only_below_zero.append(mon_decr_only_below_zero[i2])
    
    if boolean == False:
        mon_decr_only_only_below_zero.append(mon_decr_only_below_zero[i2])
    
    i2= i2+1

mon_decr_and_stable_only_below_zero

['EX_csn_e',
 'EX_glcur_e',
 'EX_gln__L_e',
 'EX_glyc2p_e',
 'EX_acgam1p_e',
 'EX_alaala_e',
 'EX_arg__L_e',
 'EX_orn_e',
 'EX_psclys_e',
 'EX_r5p_e',
 'EX_fald_e',
 'EX_frulys_e',
 'EX_fruur_e',
 'EX_g1p_e',
 'EX_gal_bD_e',
 'EX_gal1p_e',
 'EX_galctn__D_e',
 'EX_galctn__L_e',
 'EX_gam6p_e',
 'FUMt2_3pp',
 'DKMPPD']

In [492]:
i2=0
mon_decr_and_stable_both_positive_and_negative= list()
mon_decr_only_both_positive_and_negative= list()

while i2< len(mon_decr_both_positive_and_negative):  #this code separates reactions that may have flat points from reactions that always decrease for both positive and negative flux values
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(mon_decr_both_positive_and_negative[i2]))
    i=0
    l= list()
    
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    boolean = csvFile2["Biomass"].duplicated().any()
    
    if boolean == True:
        mon_decr_and_stable_both_positive_and_negative.append(mon_decr_both_positive_and_negative[i2])
    
    if boolean == False:
        mon_decr_only_both_positive_and_negative.append(mon_decr_both_positive_and_negative[i2])
    
    i2= i2+1

mon_decr_and_stable_both_positive_and_negative

['NADTRHD',
 'CYTBDpp',
 'PPA2',
 'ASPt2_2pp',
 'ALAALAD',
 'NTP3',
 'NTP10',
 'NADPHQR2',
 'CRNDabcpp',
 'NTPP6',
 'ASPT',
 'MG2tpp',
 'NAt3_2pp',
 'CD2abcpp',
 'VALabcpp',
 'EX_glcn_e',
 'TTRCYCtpp',
 'ASPt2_3pp',
 'CRNabcpp',
 'NADH10',
 'PROabcpp',
 'THRabcpp',
 'ARBt3ipp',
 'NTPP3',
 'LCTSt3ipp',
 'EX_glyclt_e',
 'DOXRBCNtpp',
 'ILEabcpp',
 'MDH3',
 'ORNabcpp',
 'MDH2',
 'NADH9',
 'THRA2',
 'NI2abcpp',
 'THIORDXi',
 'NTPP2',
 'MALt2_3pp',
 'COBALT2abcpp',
 'NTPP8',
 'G3PD5',
 'DARBabcpp',
 'PTRCabcpp',
 'ZN2abcpp',
 'NTP1',
 'NAt3_1p5pp',
 'NTPP5',
 'CMtpp',
 'NTPP1',
 'GUAt2pp',
 'LDH_D2',
 'RFAMPtpp',
 'ASNabcpp',
 'FUSAtpp',
 'G3PD6',
 'ARBabcpp',
 'FE2abcpp',
 'ARGabcpp',
 'HPYRRy',
 'EX_hxan_e',
 'ALDD2y',
 'GTHPi',
 'NADPHQR3',
 'NTPP7',
 'SUCCt2_3pp',
 'IDOND2',
 'MINCYCtpp',
 'CYTBD2pp',
 'NADH5',
 'G3PD7',
 'XANt2pp',
 'HISabcpp',
 'ALATA_D2',
 'LEUabcpp',
 'NTPP4',
 'FE2t2pp',
 'NOVBCNtpp',
 'ATPM',
 'CTBTabcpp',
 'ALDD2x',
 'NTP5',
 'GLUabcpp',
 'ALAabcpp',
 'LYSabcpp',

In [493]:
i2=0
mon_incr_and_stable_only_above_zero= list()
mon_incr_only_only_above_zero= list()

while i2< len(mon_incr_only_above_zero):  #this code separates reactions that may have flat points from reactions that always increase above zero
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(mon_incr_only_above_zero[i2]))
    i=0
    l= list()
    
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    boolean = csvFile2["Biomass"].duplicated().any()
    
    if boolean == True:
        mon_incr_and_stable_only_above_zero.append(mon_incr_only_above_zero[i2])
    
    if boolean == False:
        mon_incr_only_only_above_zero.append(mon_incr_only_above_zero[i2])
    
    i2= i2+1

mon_incr_and_stable_only_above_zero

[]

In [494]:
i2=0
mon_incr_and_stable_only_below_zero= list()
mon_incr_only_only_below_zero= list()

while i2< len(mon_incr_only_below_zero):  #this code separates reactions that may have flat points from reactions that always increase below zero
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(mon_incr_only_below_zero[i2]))
    i=0
    l= list()
    
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    boolean = csvFile2["Biomass"].duplicated().any()
    
    if boolean == True:
        mon_incr_and_stable_only_below_zero.append(mon_incr_only_below_zero[i2])
    
    if boolean == False:
        mon_incr_only_only_below_zero.append(mon_incr_only_below_zero[i2])
    
    i2= i2+1

mon_incr_and_stable_only_below_zero

['DMSOR1',
 'DMSOR1pp',
 'DMSOR2',
 'DMSOR2pp',
 'DSBAO2',
 'HYD2pp',
 'HYD3pp',
 'NO3R1pp',
 'QMO2',
 'TMAOR1',
 'TMAOR1pp',
 'TMAOR2',
 'TMAOR2pp',
 'MECDPDH2']

In [495]:
i2=0
mon_incr_and_stable= list()
mon_incr_only= list()

while i2< len(mon_incr):  #this code separates reactions that may have flat points from reactions that always increase
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(mon_incr[i2]))
    i=0
    l= list()
    
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    boolean = csvFile2["Biomass"].duplicated().any()
    
    if boolean == True:
        mon_incr_and_stable.append(mon_incr[i2])
    
    if boolean == False:
        mon_incr_only.append(mon_incr[i2])
    
    i2= i2+1


set1= set(mon_incr_and_stable)
set2= set(mon_incr_and_stable_only_above_zero)
set3= set(mon_incr_and_stable_only_below_zero)
mon_incr_and_stable_both_positive_and_negative= list(set1-set2-set3)

set4= set(mon_incr_only)
set5= set(mon_incr_only_only_above_zero)
set6= set(mon_incr_only_only_below_zero)
mon_incr_only_both_positive_and_negative= list(set4-set5-set6)

mon_incr_and_stable_both_positive_and_negative   #finds increasing and stable reactions for both positive and negative flux values
mon_incr_only_both_positive_and_negative         #finds increasing only reactions for both positive and negative flux values

['GLCptspp']

In [496]:
i=0
a= list()
b= list()
c= list()
d= list()
e= list()
actually_increasing_and_stable_below_zero_and_increasing_above_zero= list()
actually_stable_below_zero_and_increasing_above_zero= list()

while i < len(mon_decr_and_stable_both_positive_and_negative):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(mon_decr_and_stable_both_positive_and_negative[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    boolean = csvFile2["Biomass"].duplicated().any()
    
    
    n2=100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    boolean2 = csvFile3["Biomass"].duplicated().any()
    
    if boolean == True and boolean2 == False:
        a.append(mon_decr_and_stable_both_positive_and_negative[i])
        
        if len(csvFile2['Biomass'].unique()) > 1:
            actually_increasing_and_stable_below_zero_and_increasing_above_zero.append(mon_decr_and_stable_both_positive_and_negative[i])
        
        if len(csvFile2['Biomass'].unique()) == 1:
            actually_stable_below_zero_and_increasing_above_zero.append(mon_decr_and_stable_both_positive_and_negative[i])
    
    if boolean == False and boolean2 == False:
        b.append(mon_decr_and_stable_both_positive_and_negative[i])
    
    if boolean == False and boolean2 == True:
        c.append(mon_decr_and_stable_both_positive_and_negative[i])

    if boolean == True:
        d.append(mon_decr_and_stable_both_positive_and_negative[i])
    
    if boolean2 == True:
        e.append(mon_decr_and_stable_both_positive_and_negative[i])
        
    i=i+1

actually_increasing_and_stable_below_zero_and_increasing_above_zero

['NADTRHD',
 'CYTBDpp',
 'PPA2',
 'ASPt2_2pp',
 'ALAALAD',
 'NTP3',
 'NTP10',
 'NADPHQR2',
 'CRNDabcpp',
 'NTPP6',
 'ASPT',
 'MG2tpp',
 'NAt3_2pp',
 'CD2abcpp',
 'VALabcpp',
 'EX_glcn_e',
 'TTRCYCtpp',
 'ASPt2_3pp',
 'CRNabcpp',
 'NADH10',
 'PROabcpp',
 'THRabcpp',
 'ARBt3ipp',
 'NTPP3',
 'LCTSt3ipp',
 'EX_glyclt_e',
 'DOXRBCNtpp',
 'ILEabcpp',
 'MDH3',
 'ORNabcpp',
 'MDH2',
 'NADH9',
 'THRA2',
 'NI2abcpp',
 'THIORDXi',
 'NTPP2',
 'MALt2_3pp',
 'COBALT2abcpp',
 'NTPP8',
 'G3PD5',
 'DARBabcpp',
 'PTRCabcpp',
 'ZN2abcpp',
 'NTP1',
 'NAt3_1p5pp',
 'NTPP5',
 'CMtpp',
 'NTPP1',
 'GUAt2pp',
 'LDH_D2',
 'RFAMPtpp',
 'ASNabcpp',
 'FUSAtpp',
 'G3PD6',
 'ARBabcpp',
 'FE2abcpp',
 'ARGabcpp',
 'HPYRRy',
 'EX_hxan_e',
 'ALDD2y',
 'GTHPi',
 'NADPHQR3',
 'NTPP7',
 'SUCCt2_3pp',
 'IDOND2',
 'MINCYCtpp',
 'CYTBD2pp',
 'NADH5',
 'G3PD7',
 'XANt2pp',
 'HISabcpp',
 'LEUabcpp',
 'NTPP4',
 'FE2t2pp',
 'NOVBCNtpp',
 'ATPM',
 'CTBTabcpp',
 'ALDD2x',
 'NTP5',
 'GLUabcpp',
 'ALAabcpp',
 'LYSabcpp',
 'EX_cys__L_

In [497]:
i=0
a= list()
b= list()
c= list()
d= list()
e= list()
actually_decreasing_only_below_zero_and_increasing_and_stable_above_zero= list()
actually_decreasing_only_below_zero_and_stable_above_zero= list()

while i < len(mon_incr_and_stable_both_positive_and_negative):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(mon_incr_and_stable_both_positive_and_negative[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    boolean = csvFile2["Biomass"].duplicated().any()
    
    
    n2=100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    boolean2 = csvFile3["Biomass"].duplicated().any()
    
    if boolean == True and boolean2 == False:
        a.append(mon_incr_and_stable_both_positive_and_negative[i])
  
    if boolean == False and boolean2 == False:
        b.append(mon_incr_and_stable_both_positive_and_negative[i])
    
    if boolean == False and boolean2 == True:
        c.append(mon_incr_and_stable_both_positive_and_negative[i])
        
        if len(csvFile3['Biomass'].unique()) > 1:
            actually_decreasing_only_below_zero_and_increasing_and_stable_above_zero.append(mon_incr_and_stable_both_positive_and_negative[i])
        
        if len(csvFile3['Biomass'].unique()) == 1:
            actually_decreasing_only_below_zero_and_stable_above_zero.append(mon_incr_and_stable_both_positive_and_negative[i])
        
    if boolean == True:
        d.append(mon_incr_and_stable_both_positive_and_negative[i])
    
    if boolean2 == True:
        e.append(mon_incr_and_stable_both_positive_and_negative[i])
        
    i=i+1

actually_decreasing_only_below_zero_and_increasing_and_stable_above_zero

['DURAD']

In [498]:
set1= set(diff_indecr_only_above_zero)
set2= set(mon_decr_only_above_zero)
set3= set(mon_incr_only_above_zero)
inANDdecr_only_above_zero= list(set1-set2-set3)  #finds increasing and decreasing reactions above zero
inANDdecr_only_above_zero

set1= set(diff_indecr_only_below_zero)
set2= set(mon_decr_only_below_zero)
set3= set(mon_incr_only_below_zero)
inANDdecr_only_below_zero= list(set1-set2-set3)  #finds increasing and decreasing reactions below zero
inANDdecr_only_below_zero

set1= set(diff_indecr)
set2= set(diff_indecr_only_above_zero)
set3= set(diff_indecr_only_below_zero)
diff_indecr_both_positive_and_negative= list(set1-set2-set3)

set1= set(diff_indecr_both_positive_and_negative)
set2= set(mon_decr_both_positive_and_negative)
set3= set(mon_incr_both_positive_and_negative)
inANDdecr_both_positive_and_negative= list(set1-set2-set3)   #finds increasing and decreasing reactions for both positive and negative flux values
inANDdecr_both_positive_and_negative

['EAR141x',
 'GDPTPDP',
 'MALt2_2pp',
 'CYSDS',
 'APH141',
 'TPI',
 'EX_alltn_e',
 'F6PA',
 'CITL',
 'PIuabcpp',
 'ZN2t3pp',
 'CLPNH120pp',
 'ETHAt2pp',
 'FACOAE100',
 'PUNP2',
 'PDH',
 '2AGPGAT161',
 'MNt2pp',
 'PGPP161',
 'PYK',
 'NTD2',
 'CYTD',
 'FTHFD',
 'RNDR4b',
 'LYSt2pp',
 'PPK2',
 'PROD2',
 'DURIK1',
 'DALAt2pp',
 'G3PGabcpp',
 'EX_ura_e',
 'CDAPPA141',
 'EX_asn__L_e',
 'EX_4abut_e',
 'ORNDC',
 'EAR100x',
 'FLVRx',
 'CDAPPA161',
 'FACOAE80',
 'GDPMNH',
 'MDDEP3pp',
 'CDAPPA120',
 'RNTR3c2',
 'ENTERES',
 'FEENTERR2',
 'LPLIPAL2E181',
 'NDPK7',
 'CPGNR1',
 'GNK',
 'NADH17pp',
 'PGPP120',
 '2AGPGAT180',
 'PYNP2r',
 'CYTDK2',
 'IDOND',
 'CYTBO3_4pp',
 'RNDR2b',
 'GDPDPK',
 'CO2tex',
 'FBA',
 'FEENTERR3',
 'MALS',
 'GLUDy',
 'FE2tpp',
 'FEENTERR1',
 'APH120',
 'FORt2pp',
 'CDAPPA160',
 'GARFT',
 'THRt4pp',
 'NADH18pp',
 'CO2tpp',
 'PA141abcpp',
 'TKT2',
 'DCTPD',
 'H2SO',
 'CLPNH141pp',
 'TRE6PH',
 '2AGPEAT161',
 'FDH5pp',
 'EAR121x',
 'AKGDH',
 '4PEPTabcpp',
 'APH140',
 'NI2t3pp'

In [499]:
i= 0
r_id = inANDdecr_both_positive_and_negative[i]
l4= list()
l5= list()
l6= list()
l7= list()

while i< len(inANDdecr_both_positive_and_negative):   #separates inANDdecr_both_positive_and_negative reactions that actually always increase/decrease from the ones that don't
    
    r_id = inANDdecr_both_positive_and_negative[i]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    

    l= list()
    n2=100

    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    my_series = csvFile2['Biomass'].squeeze()
        
    if my_series.is_monotonic_decreasing ==True:
        l4.append(r_id)    #l4 contains reactions that have decreasing biomass values for positive flux values
    elif my_series.is_monotonic ==True:
        l5.append(r_id)    #l5 contains reactions that have increasing biomass values for positive flux values
    
    i=i+1

l4

['EAR141x',
 'GDPTPDP',
 'MALt2_2pp',
 'CYSDS',
 'APH141',
 'TPI',
 'EX_alltn_e',
 'F6PA',
 'CITL',
 'PIuabcpp',
 'ZN2t3pp',
 'CLPNH120pp',
 'ETHAt2pp',
 'FACOAE100',
 'PUNP2',
 'PDH',
 '2AGPGAT161',
 'MNt2pp',
 'PGPP161',
 'PYK',
 'NTD2',
 'CYTD',
 'FTHFD',
 'RNDR4b',
 'LYSt2pp',
 'PPK2',
 'PROD2',
 'DURIK1',
 'DALAt2pp',
 'G3PGabcpp',
 'EX_ura_e',
 'CDAPPA141',
 'EX_asn__L_e',
 'EX_4abut_e',
 'ORNDC',
 'EAR100x',
 'FLVRx',
 'CDAPPA161',
 'FACOAE80',
 'GDPMNH',
 'MDDEP3pp',
 'CDAPPA120',
 'RNTR3c2',
 'ENTERES',
 'FEENTERR2',
 'LPLIPAL2E181',
 'NDPK7',
 'CPGNR1',
 'GNK',
 'NADH17pp',
 'PGPP120',
 '2AGPGAT180',
 'PYNP2r',
 'CYTDK2',
 'IDOND',
 'RNDR2b',
 'GDPDPK',
 'CO2tex',
 'FBA',
 'FEENTERR3',
 'MALS',
 'GLUDy',
 'FE2tpp',
 'FEENTERR1',
 'APH120',
 'FORt2pp',
 'CDAPPA160',
 'GARFT',
 'THRt4pp',
 'NADH18pp',
 'CO2tpp',
 'PA141abcpp',
 'TKT2',
 'DCTPD',
 'H2SO',
 'CLPNH141pp',
 'TRE6PH',
 '2AGPEAT161',
 'FDH5pp',
 'EAR121x',
 'AKGDH',
 '4PEPTabcpp',
 'APH140',
 'NI2t3pp',
 'RNTR1c2',
 

In [500]:
i= 0
r_id = inANDdecr_both_positive_and_negative[i]
l7= list()

while i< len(inANDdecr_both_positive_and_negative):   #separates inANDdecr_both_positive_and_negative reactions that actually always increase/decrease from the ones that don't
    
    r_id = inANDdecr_both_positive_and_negative[i]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        
    l= list()
    n2=0

    while n2 < 101:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    my_series = csvFile2['Biomass'].squeeze()
        
    if my_series.is_monotonic_decreasing ==True:
        l6.append(r_id)    #l6 contains reactions that have decreasing biomass values for negative flux values
    elif my_series.is_monotonic ==True:
        l7.append(r_id)    #l7 contains reactions that have increasing biomass values for negative flux values
    
    i=i+1

l7


['EAR141x',
 'GDPTPDP',
 'MALt2_2pp',
 'CYSDS',
 'APH141',
 'TPI',
 'F6PA',
 'CITL',
 'ZN2t3pp',
 'CLPNH120pp',
 'ETHAt2pp',
 'FACOAE100',
 'PUNP2',
 'PDH',
 '2AGPGAT161',
 'PGPP161',
 'PYK',
 'NTD2',
 'CYTD',
 'FTHFD',
 'RNDR4b',
 'LYSt2pp',
 'PPK2',
 'PROD2',
 'DURIK1',
 'DALAt2pp',
 'G3PGabcpp',
 'EX_ura_e',
 'CDAPPA141',
 'ORNDC',
 'EAR100x',
 'FLVRx',
 'CDAPPA161',
 'FACOAE80',
 'GDPMNH',
 'MDDEP3pp',
 'CDAPPA120',
 'RNTR3c2',
 'ENTERES',
 'FEENTERR2',
 'LPLIPAL2E181',
 'NDPK7',
 'CPGNR1',
 'GNK',
 'NADH17pp',
 'PGPP120',
 '2AGPGAT180',
 'PYNP2r',
 'CYTDK2',
 'IDOND',
 'CYTBO3_4pp',
 'RNDR2b',
 'GDPDPK',
 'FBA',
 'FEENTERR3',
 'MALS',
 'FE2tpp',
 'FEENTERR1',
 'APH120',
 'FORt2pp',
 'CDAPPA160',
 'GARFT',
 'THRt4pp',
 'NADH18pp',
 'PA141abcpp',
 'TKT2',
 'DCTPD',
 'CLPNH141pp',
 'TRE6PH',
 '2AGPEAT161',
 'FDH5pp',
 'EAR121x',
 'AKGDH',
 '4PEPTabcpp',
 'APH140',
 'NI2t3pp',
 'RNTR1c2',
 'GLYCTO3',
 'AGM3PApp',
 'ARGDC',
 'EX_val__L_e',
 'G3PT',
 '2AGPGAT181',
 'FMNRx',
 '2AGPEAT160

In [501]:
D1= pd.DataFrame (l4, columns = ['Reaction names'])
D2= pd.DataFrame (l7, columns = ['Reaction names'])
b= D1.iloc[:, 0]
c= D2.iloc[:, 0]

union = pd.Series(np.union1d(b, c))

set1= set(inANDdecr_both_positive_and_negative)
set2= set(union)
intersect = pd.Series(np.intersect1d(b, c))

inANDdecr_both_positive_and_negative_left= set1-set2
inANDdecr_both_positive_and_negative_left


set()

In [502]:
set3= set(l4)
set4= set(l7)

i=0
lnew=list()

while i<len(l4):
    if (l4[i] in l7) == False:
        lnew.append(l4[i])
    i=i+1

lnew

['EX_alltn_e',
 'PIuabcpp',
 'MNt2pp',
 'EX_asn__L_e',
 'EX_4abut_e',
 'CO2tex',
 'GLUDy',
 'CO2tpp',
 'H2SO',
 'PPS',
 'CAT',
 'EX_idon__L_e',
 'EX_etoh_e',
 'ZNabcpp',
 'EX_pro__L_e',
 'EX_glyc_e',
 'GLYCDx',
 'EX_ade_e',
 'EX_for_e',
 'EX_ala__D_e',
 'EX_gua_e',
 'THD2pp',
 'EX_succ_e',
 'H2Otex',
 'PPCK',
 'EX_xan_e',
 'H2Otpp',
 'PGK',
 'NI2uabcpp',
 'FBP',
 'PGM',
 'EX_12ppd__S_e',
 'FE3abcpp',
 'FLDR2',
 'EX_ac_e',
 'Kabcpp',
 'PPA',
 'EX_glyc__R_e',
 'ARGDCpp']

In [503]:
i=0
lnew2=list()

while i<len(l7):
    if (l7[i] in l4) == False:
        lnew2.append(l7[i])
    i=i+1

lnew2

['CYTBO3_4pp',
 'EX_h2o_e',
 'MDH',
 'GAPD',
 'ATPS4rpp',
 'ENO',
 'EX_co2_e',
 'SUCDi',
 'FUM']

In [504]:
i2=0
inANDdecr_and_stable_only_above_zero= list()

while i2< len(inANDdecr_only_above_zero):  #this code finds increasing and decreasng and stable reactions above zero
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(inANDdecr_only_above_zero[i2]))
    i=0
    l= list()
    
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    i=0
    
    while i < len(csvFile2)-1:
        if csvFile2.iloc[i,  0] == csvFile2.iloc[i+1,  0] and inANDdecr_only_above_zero[i2] not in inANDdecr_and_stable_only_above_zero:
            inANDdecr_and_stable_only_above_zero.append(inANDdecr_only_above_zero[i2])
            
        i=i+1
        
    i2= i2+1

inANDdecr_and_stable_only_above_zero


['O2tex']

In [505]:
i2=0
inANDdecr_and_stable_only_below_zero= list()

while i2< len(inANDdecr_only_below_zero):  #this code finds increasing and decreasng and stable reactions below zero
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(inANDdecr_only_below_zero[i2]))
    i=0
    l= list()
    
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    i=0
    
    while i < len(csvFile2)-1:
        if csvFile2.iloc[i,  0] == csvFile2.iloc[i+1,  0] and inANDdecr_only_below_zero[i2] not in inANDdecr_and_stable_only_below_zero:
            inANDdecr_and_stable_only_below_zero.append(inANDdecr_only_below_zero[i2])
            
        i=i+1
        
    i2= i2+1

inANDdecr_and_stable_only_below_zero

['NTRIR2x',
 'EX_glc__D_e',
 'EX_fru_e',
 'EX_lyx__L_e',
 'EX_udpg_e',
 'EX_ethso3_e',
 'EX_o2_e',
 'EX_acmana_e',
 'NO3R1bpp',
 'EX_man_e',
 'EX_agm_e',
 'EX_galur_e',
 'EX_xyl__D_e',
 'EX_uacgam_e',
 'EX_ppa_e',
 'EX_acgam_e',
 'EX_arab__L_e',
 'EX_manglyc_e',
 'EX_gam_e',
 'EX_gal_e',
 'EX_udpgal_e',
 'EX_xylu__L_e',
 'EX_cynt_e',
 'EX_ppal_e']

In [506]:
i2=0
inANDdecr_and_stable_both_positive_and_negative= list()

while i2< len(inANDdecr_both_positive_and_negative):  #this code finds increasing and decreasng and stable reactions for both positive and negative flux values
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(inANDdecr_both_positive_and_negative[i2]))
    i=0
    l= list()
    
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    i=0
    
    while i < len(csvFile2)-1:
        if csvFile2.iloc[i,  0] == csvFile2.iloc[i+1,  0] and inANDdecr_both_positive_and_negative[i2] not in inANDdecr_and_stable_both_positive_and_negative:
            inANDdecr_and_stable_both_positive_and_negative.append(inANDdecr_both_positive_and_negative[i2])
            
        i=i+1
        
    i2= i2+1

inANDdecr_and_stable_both_positive_and_negative


['CITL',
 'PIuabcpp',
 'PDH',
 'MNt2pp',
 'NADH17pp',
 'NADH18pp',
 'H2SO',
 'FDH5pp',
 'GLYCTO3',
 'ASNN',
 'ZNabcpp',
 'ACALD',
 'GLYCTO4',
 'EX_glyc_e',
 'EX_h2o_e',
 'PGI',
 'EX_for_e',
 'DHORD5',
 'EX_gua_e',
 'THD2pp',
 'H2Otex',
 'EX_xan_e',
 'ALATA_L2',
 'NI2uabcpp',
 'FRD2',
 'SUCDi',
 'EX_ac_e',
 'ME2',
 'Kabcpp',
 'PPA',
 'PYRDC',
 'EX_glyc__R_e',
 'ARGDCpp']

In [507]:
"ACACT8r_copy2" in intersect
len(inANDdecr_and_stable_both_positive_and_negative_intersect)+len(inANDdecr_and_stable_both_positive_and_negative_lnew)+len(inANDdecr_and_stable_both_positive_and_negative_lnew2)
len(inANDdecr_and_stable_both_positive_and_negative)

33

In [508]:
i2=0
inANDdecr_and_stable_both_positive_and_negative_intersect= list()

while i2< len(intersect):  #this code finds increasing and decreasng and stable reactions for both positive and negative flux values
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(intersect[i2]))
    i=0
    l= list()
    
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    i=0
    
    while i < len(csvFile2)-1:
        if csvFile2.iloc[i,  0] == csvFile2.iloc[i+1,  0] and intersect[i2] not in inANDdecr_and_stable_both_positive_and_negative_intersect:
            inANDdecr_and_stable_both_positive_and_negative_intersect.append(intersect[i2])
            
        i=i+1
        
    i2= i2+1

inANDdecr_and_stable_both_positive_and_negative_intersect

['ACALD',
 'ALATA_L2',
 'ASNN',
 'CITL',
 'DHORD5',
 'FDH5pp',
 'FRD2',
 'GLYCTO3',
 'GLYCTO4',
 'ME2',
 'NADH17pp',
 'NADH18pp',
 'PDH',
 'PGI',
 'PYRDC']

In [509]:
i2=0
inANDdecr_and_stable_both_positive_and_negative_lnew= list()

while i2< len(lnew):  #this code finds increasing and decreasng and stable reactions for both positive and negative flux values
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(lnew[i2]))
    i=0
    l= list()
    
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    i=0
    
    while i < len(csvFile2)-1:
        if csvFile2.iloc[i,  0] == csvFile2.iloc[i+1,  0] and lnew[i2] not in inANDdecr_and_stable_both_positive_and_negative_lnew:
            inANDdecr_and_stable_both_positive_and_negative_lnew.append(lnew[i2])
            
        i=i+1
        
    i2= i2+1

inANDdecr_and_stable_both_positive_and_negative_lnew

['PIuabcpp',
 'MNt2pp',
 'H2SO',
 'ZNabcpp',
 'EX_glyc_e',
 'EX_for_e',
 'EX_gua_e',
 'THD2pp',
 'H2Otex',
 'EX_xan_e',
 'NI2uabcpp',
 'EX_ac_e',
 'Kabcpp',
 'PPA',
 'EX_glyc__R_e',
 'ARGDCpp']

In [510]:
i2=0
inANDdecr_and_stable_both_positive_and_negative_lnew2= list()

while i2< len(lnew2):  #this code finds increasing and decreasng and stable reactions for both positive and negative flux values
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(lnew2[i2]))
    i=0
    l= list()
    
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    i=0
    
    while i < len(csvFile2)-1:
        if csvFile2.iloc[i,  0] == csvFile2.iloc[i+1,  0] and lnew2[i2] not in inANDdecr_and_stable_both_positive_and_negative_lnew2:
            inANDdecr_and_stable_both_positive_and_negative_lnew2.append(lnew2[i2])
            
        i=i+1
        
    i2= i2+1

inANDdecr_and_stable_both_positive_and_negative_lnew2

['EX_h2o_e', 'SUCDi']

In [511]:
set1= set(inANDdecr_only_above_zero)
set2= set(inANDdecr_and_stable_only_above_zero)
l_left= list(set1-set2)
l_left

i2=0
inANDdecr_only_only_above_zero= list()

while i2< len(l_left):  #this code finds increasing and decreasng only reactions above zero
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(l_left[i2]))
    i=0
    l= list()
    
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    i=0
        
    while i < len(csvFile2)-1:
        
        if csvFile2.iloc[i,  0] != csvFile2.iloc[i+1,  0] and l_left[i2] not in inANDdecr_only_only_above_zero:
            inANDdecr_only_only_above_zero.append(l_left[i2])
        
        i=i+1
        
    i2= i2+1

inANDdecr_only_only_above_zero

['SERAT',
 'AICART',
 'ACONTb',
 'ACGS',
 'GK1',
 'NH4tpp',
 'ARGSS',
 'HSK',
 'CTPS2',
 'HCO3E',
 'PSSA160',
 'DASYN160',
 'ACODA',
 'MCOATA',
 'UAMAS',
 'TMPK',
 'GLUPRT',
 'PRFGS',
 'UAMAGS',
 'SADT2',
 'ACPPAT160',
 'GMPS2',
 'PSD160',
 'PMPK',
 'UAAGDS',
 'ACGK',
 'ARGSL',
 'AGPAT161',
 'THRS',
 'NDPK4',
 'UGMDDS',
 'NNATr',
 'NH4tex',
 'APG3PAT161',
 'P5CR',
 'ADSL1r',
 'BPNT',
 'OCBT',
 'PSSA161',
 'ACPPAT161',
 'UAGCVT',
 'ADSL2r',
 'SULR',
 'PSD161',
 'AIRC2',
 'CU2tpp',
 'AGPAT160',
 'ALAALAr',
 'G5SADs',
 'CS',
 'COBALT2tpp',
 'ADSS',
 'DASYN161',
 'O2tpp',
 'UMPK',
 'GLNS',
 'PRAIS',
 '3OAS140',
 'PE160abcpp',
 'UAPGR',
 'ALAR',
 'DTMPK',
 'PE161abcpp',
 'UAGPT3',
 'PAPPT3',
 'ADSK',
 'CYSS',
 '3OAR140',
 'PRASCSi',
 'ACONTa',
 'G1PACT',
 'APG3PAT160',
 'ACCOAC',
 'UDCPDP',
 'SO4tex',
 'NDPK2',
 'PRAGSr',
 'NADK',
 'UAGDP',
 'NADS1',
 'ICDHyr',
 'THRD_L',
 'ASPK',
 'GF6PTA']

In [512]:
set1= set(inANDdecr_only_below_zero)
set2= set(inANDdecr_and_stable_only_below_zero)
l_left= list(set1-set2)
l_left

i2=0
inANDdecr_only_only_below_zero= list()

while i2< len(l_left):  #this code finds increasing and decreasng only reactions below zero
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(l_left[i2]))
    i=0
    l= list()
    
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    i=0
        
    while i < len(csvFile2)-1:
        
        if csvFile2.iloc[i,  0] != csvFile2.iloc[i+1,  0] and l_left[i2] not in inANDdecr_only_only_below_zero:
            inANDdecr_only_only_below_zero.append(l_left[i2])
        
        i=i+1
        
    i2= i2+1

inANDdecr_only_only_below_zero

['EX_LalaDgluMdap_e',
 'EX_melib_e',
 'EX_LalaDglu_e',
 'EX_ocdca_e',
 'EX_rib__D_e',
 'IMPC',
 'OPHHX3',
 'EX_acmum_e',
 'EX_but_e',
 'EX_ddca_e',
 'OMMBLHX3',
 'EX_glcr_e',
 'EX_ocdcea_e',
 'EX_octa_e',
 'EX_26dap__M_e',
 'EX_23dappa_e',
 'EX_all__D_e',
 'EX_pppn_e',
 'EX_23ccmp_e',
 'AGPR',
 'EX_gthrd_e',
 'RPI',
 'EX_fum_e',
 'ACOTA',
 'EX_LalaLglu_e',
 'EX_tartr__D_e',
 'EX_sucr_e',
 'EX_hdca_e',
 'EX_hxa_e',
 'EX_udpglcur_e',
 'EX_butso3_e',
 'EX_malt_e',
 'EX_dca_e',
 'G3PD2',
 'EX_3hpppn_e',
 'EX_3hcinnm_e',
 'GLUR',
 'EX_tartr__L_e',
 'EX_galct__D_e',
 'EX_progly_e',
 'OMPHHX3',
 'EX_udpacgal_e',
 'EX_chtbs_e',
 'EX_hdcea_e',
 'EX_acnam_e',
 'HSDy',
 'EX_anhgm_e',
 'EX_nh4_e',
 'EX_ttdcea_e',
 'EX_ttdca_e',
 'EX_ascb__L_e',
 'ASPTA',
 'AIRC3',
 'ASAD',
 'EX_LalaDgluMdapDala_e',
 'EX_sbt__D_e',
 'EX_cgly_e',
 'EX_mnl_e',
 'EX_ptrc_e',
 'EX_fe3dcit_e',
 'PGAMT',
 'EX_galt_e',
 'EX_so4_e']

In [513]:
set1= set(inANDdecr_both_positive_and_negative)
set2= set(inANDdecr_and_stable_both_positive_and_negative)
l_left= list(set1-set2)
l_left

i2=0
inANDdecr_only_both_positive_and_negative= list()

while i2< len(l_left):  #this code finds increasing and decreasng only reactions for both positive and negative flux values
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(l_left[i2]))
    i=0
    l= list()
    
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    i=0
        
    while i < len(csvFile2)-1:
        
        if csvFile2.iloc[i,  0] != csvFile2.iloc[i+1,  0] and l_left[i2] not in inANDdecr_only_both_positive_and_negative:
            inANDdecr_only_both_positive_and_negative.append(l_left[i2])
        
        i=i+1
        
    i2= i2+1

inANDdecr_only_both_positive_and_negative


['EAR141x',
 'GDPTPDP',
 'MALt2_2pp',
 'CYSDS',
 'APH141',
 'TPI',
 'EX_alltn_e',
 'F6PA',
 'ZN2t3pp',
 'CLPNH120pp',
 'ETHAt2pp',
 'FACOAE100',
 'PUNP2',
 '2AGPGAT161',
 'PGPP161',
 'PYK',
 'NTD2',
 'CYTD',
 'FTHFD',
 'RNDR4b',
 'LYSt2pp',
 'PPK2',
 'PROD2',
 'DURIK1',
 'DALAt2pp',
 'G3PGabcpp',
 'EX_ura_e',
 'CDAPPA141',
 'EX_asn__L_e',
 'EX_4abut_e',
 'ORNDC',
 'EAR100x',
 'FLVRx',
 'CDAPPA161',
 'FACOAE80',
 'GDPMNH',
 'MDDEP3pp',
 'CDAPPA120',
 'RNTR3c2',
 'ENTERES',
 'FEENTERR2',
 'LPLIPAL2E181',
 'NDPK7',
 'CPGNR1',
 'GNK',
 'PGPP120',
 '2AGPGAT180',
 'PYNP2r',
 'CYTDK2',
 'IDOND',
 'CYTBO3_4pp',
 'RNDR2b',
 'GDPDPK',
 'CO2tex',
 'FBA',
 'FEENTERR3',
 'MALS',
 'GLUDy',
 'FE2tpp',
 'FEENTERR1',
 'APH120',
 'FORt2pp',
 'CDAPPA160',
 'GARFT',
 'THRt4pp',
 'CO2tpp',
 'PA141abcpp',
 'TKT2',
 'DCTPD',
 'CLPNH141pp',
 'TRE6PH',
 '2AGPEAT161',
 'EAR121x',
 'AKGDH',
 '4PEPTabcpp',
 'APH140',
 'NI2t3pp',
 'RNTR1c2',
 'AGM3PApp',
 'PPS',
 'ARGDC',
 'EX_val__L_e',
 'G3PT',
 '2AGPGAT181',
 '

In [514]:
D= pd.DataFrame (one_at_zero, columns = ['Reaction names'])
d1= pd.DataFrame (diff_saturated_only_above_zero, columns = ['Reaction names'])
d2= pd.DataFrame (flat_only_above_zero, columns = ['Reaction names'])
d3= pd.DataFrame (flat_only_below_zero, columns = ['Reaction names'])
d4= pd.DataFrame (flat_both_positive_and_negative, columns = ['Reaction names'])
d5= pd.DataFrame (diff_saturatedFINAL_only_below_zero, columns = ['Reaction names'])
d6= pd.DataFrame (Ushaped_reactions_only_below_zero, columns = ['Reaction names'])
d7= pd.DataFrame (diff_saturatedFINAL_both_positive_and_negative, columns = ['Reaction names'])
d8= pd.DataFrame (Ushaped_reactions_both_positive_and_negative, columns = ['Reaction names'])
d9= pd.DataFrame (mon_decr_and_stable_only_above_zero, columns = ['Reaction names'])
d10= pd.DataFrame (mon_decr_only_only_above_zero, columns = ['Reaction names'])
d11= pd.DataFrame (mon_decr_and_stable_only_below_zero, columns = ['Reaction names'])
d12= pd.DataFrame (mon_decr_only_only_below_zero, columns = ['Reaction names'])
d13= pd.DataFrame (mon_incr_and_stable_only_above_zero, columns = ['Reaction names'])
d14= pd.DataFrame (mon_incr_only_only_above_zero, columns = ['Reaction names'])
d15= pd.DataFrame (mon_incr_and_stable_only_below_zero, columns = ['Reaction names'])
d16= pd.DataFrame (mon_incr_only_only_below_zero, columns = ['Reaction names'])
d17= pd.DataFrame (mon_incr_only_both_positive_and_negative, columns = ['Reaction names'])
d18= pd.DataFrame (mon_decr_only_both_positive_and_negative, columns = ['Reaction names'])
d19= pd.DataFrame (actually_increasing_and_stable_below_zero_and_increasing_above_zero, columns = ['Reaction names'])
d20= pd.DataFrame (actually_stable_below_zero_and_increasing_above_zero, columns = ['Reaction names'])
d21= pd.DataFrame (actually_decreasing_only_below_zero_and_increasing_and_stable_above_zero, columns = ['Reaction names'])
d22= pd.DataFrame (actually_decreasing_only_below_zero_and_stable_above_zero, columns = ['Reaction names'])
d23= pd.DataFrame (inANDdecr_and_stable_only_above_zero, columns = ['Reaction names'])
d24= pd.DataFrame (inANDdecr_and_stable_only_below_zero, columns = ['Reaction names'])
d25= pd.DataFrame (inANDdecr_and_stable_both_positive_and_negative_intersect, columns = ['Reaction names'])
d26= pd.DataFrame (inANDdecr_and_stable_both_positive_and_negative_lnew, columns = ['Reaction names'])
d27= pd.DataFrame (inANDdecr_and_stable_both_positive_and_negative_lnew2, columns = ['Reaction names'])
d28= pd.DataFrame (inANDdecr_only_only_above_zero, columns = ['Reaction names'])
d29= pd.DataFrame (inANDdecr_only_only_below_zero, columns = ['Reaction names'])
d30= pd.DataFrame (inANDdecr_only_both_positive_and_negative, columns = ['Reaction names'])

len(D)+len(d1)+len(d2)+len(d3)+len(d4)+len(d5)+len(d6)+len(d7)+len(d8)+len(d9)+len(d10)+len(d11)+len(d12)+len(d13)+len(d14)+len(d15)+len(d16)+len(d17)+len(d18)+len(d19)+len(d20)+len(d21)+len(d22)+len(d23)+len(d24)+len(d25)+len(d26)+len(d27)+len(d28)+len(d29)+len(d30)

2727

In [517]:
D.to_csv("one feasible value at zero reactions.csv")
d1.to_csv("diff_saturated only above zero.csv")
d2.to_csv("flat only above zero.csv")
d3.to_csv("flat only below zero.csv")
d4.to_csv("flat both positive and negative.csv")
d5.to_csv("diff_saturated only below zero.csv")
d6.to_csv("Ushaped_reactions only below zero.csv")
d7.to_csv("diff_saturated both positive and negative.csv")
d8.to_csv("Ushaped both positive and negative.csv")
d9.to_csv("decreasing and stable only above zero reactions.csv")
d10.to_csv("decreasing only above zero reactions.csv")
d11.to_csv("increasing and stable only below zero reactions.csv")
d12.to_csv("increasing only below zero reactions.csv")
d13.to_csv("monotonically increasing and stable only above zero reactions.csv")
d14.to_csv("monotonically increasing only above zero reactions.csv")
d15.to_csv("decreasing and stable only below zero reactions.csv")
d16.to_csv("decreasing only below zero reactions.csv")
d17.to_csv("decreasing only below zero and increasing only above zero reactions.csv")
d18.to_csv("increasing only below zero and decreasing only above zero reactions.csv")
d19.to_csv("increasing and stable only below zero and increasing only above zero reactions.csv")
d20.to_csv("stable only below zero and increasing only above zero reactions.csv")
d21.to_csv("decreasing only below zero and increasing and stable only above zero reactions.csv")
d22.to_csv("decreasing only below zero and stable only above zero reactions.csv")
d23.to_csv("increasing and decreasing and stable only above zero reactions.csv")
d24.to_csv("increasing and decreasing and stable only below zero reactions.csv")
d25.to_csv("decreasing and stable both positive and negative reactions.csv")
d26.to_csv("increasing orand decreasing or both and stable below zero and decreasing above zero.csv")
d27.to_csv("decreasing below zero and increasing or decreasing or both and stable above zero.csv")
d28.to_csv("increasing and decreasing only above zero.csv")
d29.to_csv("increasing and decreasing only below zero.csv")
d30.to_csv("increasing and decreasing both positive and negative.csv")


In [470]:
len(mon_decr_only_only_above_zero) + len(mon_decr_only_only_below_zero) + len(mon_incr_only_only_above_zero) + len(mon_incr_only_only_below_zero) + len(flat_only_above_zero) + len(diff_saturated_only_above_zero) + len(D_at_zero) + len(flat_only_below_zero) + len(flat_both_positive_and_negative)+len(diff_saturatedFINAL_only_below_zero)+len(Ushaped_reactions_only_below_zero)+len(diff_saturatedFINAL_both_positive_and_negative)+len(Ushaped_reactions_both_positive_and_negative)+len(mon_decr_and_stable_only_above_zero)+len(mon_decr_and_stable_only_below_zero)+len(mon_incr_and_stable_only_above_zero)+len(mon_incr_and_stable_only_below_zero)+len(mon_incr_only_both_positive_and_negative)+len(mon_decr_only_both_positive_and_negative)+len(actually_increasing_and_stable_below_zero_and_increasing_above_zero)+len(actually_stable_below_zero_and_increasing_above_zero)+len(actually_decreasing_only_below_zero_and_increasing_and_stable_above_zero)+len(actually_decreasing_only_below_zero_and_stable_above_zero)+len(inANDdecr_and_stable_only_above_zero)+len(inANDdecr_and_stable_only_below_zero)+len(inANDdecr_and_stable_both_positive_and_negative_intersect)+len(inANDdecr_and_stable_both_positive_and_negative_lnew)+len(inANDdecr_and_stable_both_positive_and_negative_lnew2)+len(inANDdecr_only_only_above_zero)+len(inANDdecr_only_only_below_zero)+len(inANDdecr_only_both_positive_and_negative)

2727

In [474]:
len(inANDdecr_and_stable_both_positive_and_negative)

33

In [475]:
len(inANDdecr_and_stable_both_positive_and_negative_intersect)+len(inANDdecr_and_stable_both_positive_and_negative_lnew)+len(inANDdecr_and_stable_both_positive_and_negative_lnew2)

33